In [1]:
import os
os.chdir("/dust3r")
print(os.getcwd())

/dust3r


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import json
import open3d as o3d

from dust3r.inference import inference_with_mask, create_gaussian_kernel
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode
from dust3r.cloud_opt.base_opt import global_alignment_loop
from masked_dust3r.scripts.utils.math import *
from masked_dust3r.scripts.utils.image import *


DATA_PATH = "/dust3r/masked_dust3r/data/jackal_training_data_0"
IMG_FILE_EXTENSION = ".png"
MASK_FILE_EXTENSION = ".png"

INIT_FRAMES = 50
NEW_FRAMES = 10
PREVIOUS_FRAMES = 40
TOTAL_FRAMES = 300

INIT_WEIGHT_FOCAL = 0.1
INIT_WEIGHT_Z = 0.1 
INIT_WEIGHT_ROT = 0.1
INIT_WEIGHT_TRANS_SMOOTHNESS = 0.0001 
INIT_WEIGHT_ROT_SMOOTHNESS = 0.001 * 0

NEW_WEIGHT_FOCAL = 0.1
NEW_WEIGHT_Z = 0.1
NEW_WEIGHT_ROT = 0.1
NEW_WEIGHT_TRANS_SMOOTHNESS = 0.0001
NEW_WEIGHT_ROT_SMOOTHNESS = 0.00001

USE_COMMON_INTRINSICS = True
IS_FOCAL_FIXED = True
FOCAL_LENGTH = 4.74

device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


In [3]:
GAUSSIAN_SIGMA = 51.0
SIZE = int(GAUSSIAN_SIGMA * 3)

kernel = create_gaussian_kernel(SIZE, GAUSSIAN_SIGMA).to(device)

SIZE = 11
kernel = torch.ones(SIZE, SIZE).to(device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# Load the model

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>


In [5]:
images_array = []
masks_array = []

for i in range(0,25):
    images_array.append(os.path.join(DATA_PATH,"masked_images/{}{}".format(i,IMG_FILE_EXTENSION)))
    masks_array.append(os.path.join(DATA_PATH,"masks/{}{}".format(i,MASK_FILE_EXTENSION)))
images = load_images(images_array, size=512, verbose=True)
_,_,H,W = images[0]["img"].shape
masks = load_masks(masks_array, H, W, device)

>> Loading a list of 25 images
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/0.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/1.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/2.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/3.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/4.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/5.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/6.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/data/jackal_training_data_0/masked_images/7.png with resolution 1280x720 --> 512x288
 - adding /dust3r/masked_dust3r/d

In [6]:
pairs = make_pairs(images, scene_graph='swin-5', prefilter=None, symmetrize=True)
output = inference_with_mask(pairs, model, device, masks, kernel, batch_size=batch_size)
del pairs

>> Inference with model on 250 image pairs


100%|██████████| 250/250 [01:19<00:00,  3.16it/s]


In [7]:
#init_scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer)
#loss = init_scene.compute_global_alignment(init="mst", niter=niter, schedule='cosine', lr=lr)

#scene = init_scene

scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PlanePointCloudOptimizer, 
                        weight_focal = INIT_WEIGHT_FOCAL,
                        weight_z = INIT_WEIGHT_Z ,
                        weight_rot = INIT_WEIGHT_ROT  ,
                        weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS,
                        weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS)
#scene.im_poses = calculate_new_params(init_scene.im_poses,device)
#scene.im_focals = init_scene.im_focals
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#averge_focal = scene.get_focals().sum().item()/len(images_array)
#fixed_focal = [averge_focal for _ in range(len(images_array))]
#mask = [True for _ in range(len(images_array))]
#scene.preset_focal(fixed_focal, mask)
#loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

#scene.weight_focal = INIT_WEIGHT_FOCAL
#scene.weight_z = INIT_WEIGHT_Z 
#scene.weight_rot = INIT_WEIGHT_ROT
#scene.weight_trans_smoothness = INIT_WEIGHT_TRANS_SMOOTHNESS 
#scene.weight_rot_smoothness = INIT_WEIGHT_ROT_SMOOTHNESS
#loss = global_alignment_loop(scene, niter=niter, schedule=schedule, lr=lr)


 init edge (17*,20*) score=1.2195663452148438
 init edge (17,19*) score=1.2064969539642334
 init edge (19,21*) score=1.2022987604141235
 init edge (17,18*) score=1.1798925399780273
 init edge (20,16*) score=1.1648982763290405
 init edge (20,22*) score=1.158280372619629
 init edge (20,23*) score=1.1558665037155151
 init edge (19,15*) score=1.1521621942520142
 init edge (21,1*) score=1.1367660760879517
 init edge (19,24*) score=1.116551160812378
 init edge (19,14*) score=1.111879825592041
 init edge (20,0*) score=1.109182357788086
 init edge (17,12*) score=1.0916748046875
 init edge (17,13*) score=1.0662122964859009
 init edge (1,6*) score=1.1510916948318481
 init edge (6,10*) score=1.1484986543655396
 init edge (6,11*) score=1.1329917907714844
 init edge (3*,6) score=1.204742193222046
 init edge (3,8*) score=1.1924866437911987
 init edge (4*,8) score=1.191866397857666
 init edge (3,7*) score=1.1806812286376953
 init edge (3,2*) score=1.1565231084823608
 init edge (3,5*) score=1.20661747

100%|██████████| 300/300 [04:47<00:00,  1.04it/s, lr=1.27413e-06 loss=0.0744462]


In [22]:
all_focal = torch.stack(list(scene.im_focals))
all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize()#.to_homogeneous()#.inverse()
tf_inv = tf.inverse()

off_z_axis = torch.tensor([]).to(device)

#print(tf.linear[:,3])

for e, (i, j) in enumerate(scene.edges):
    #print(torch.nn.functional.normalize((tf_inv[i] @ tf[j]).linear[:3],p=2, dim=0, eps=1e-12,))
    print(torch.nn.functional.normalize((tf[j] @ tf_inv[i]).linear[:3],p=2, dim=0, eps=1e-12)[2].abs())
    #off_z_axis = torch.cat((off_z_axis, 1-torch.nn.functional.normalize((tf[j] @ tf_inv[i]).linear[:3],p=2, dim=0, eps=1e-12)[2].abs().unsqueeze(0)))


tensor([ 8.2226e-07, -7.2042e-06, -5.4512e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 4.3592e-06,  2.8184e-06, -4.5695e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 1.2251e-06, -2.8411e-06, -4.5785e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 3.9716e-06,  2.0757e-06, -9.1663e-02], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([-1.8090e-06, -5.2150e-06, -6.1409e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 1.5845e-06, -4.7310e-06, -5.5031e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([-3.5628e-05, -2.8911e-05, -2.8153e+00], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([8.9784e-06, 2.5178e-05, 7.0868e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 2.0900e-05, -5.4354e-07, -8.0479e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 2.0859e-05,  1.2012e-05, -8.8571e-01], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
ten

In [9]:
print(tf[0])

RigidUnitQuat(linear=tensor([ 0.0581, -0.2086, -0.5118,  0.8313], device='cuda:0',
       grad_fn=<SelectBackward0>), translation=tensor([ 0.0921, -0.1534,  0.0082], device='cuda:0', grad_fn=<SelectBackward0>))


In [10]:
all_focal = torch.stack(list(scene.im_focals))
print((all_focal.max() - all_focal.min()) ** 2)

tensor(17.9377, device='cuda:0', grad_fn=<PowBackward0>)


In [11]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()
intrinsics = scene.get_intrinsics()

In [12]:
print(focals)

tensor([[466.8382],
        [466.8387],
        [466.8382],
        [481.8183],
        [487.4292],
        [466.8389],
        [466.8398],
        [475.3553],
        [466.8380],
        [411.5982],
        [441.2984],
        [466.8393],
        [466.8393],
        [466.8389],
        [431.4714],
        [466.8391],
        [454.9757],
        [481.8183],
        [466.8398],
        [500.6058],
        [484.1960],
        [466.8389],
        [453.1047],
        [466.8402],
        [508.6766]], device='cuda:0', grad_fn=<ExpBackward0>)


In [13]:
#Check if pointclouds folder exists
#If exists, delete all files in the folder
if os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    for file in os.listdir("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
        os.remove("{DATA_PATH}/pointclouds/{file}".format(DATA_PATH=DATA_PATH, file=file))
        
if not os.path.exists("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH)):
    os.makedirs("{DATA_PATH}/pointclouds".format(DATA_PATH=DATA_PATH))

for i in range(len(images)):
    pointcloud = pts3d[i].detach().cpu().numpy()
    pointcloud = pointcloud.reshape(-1, 3)
    color = imgs[i].reshape(-1, 3)
    confidence_mask = confidence_masks[i].detach().cpu().numpy()
    confidence_mask = confidence_mask.reshape(-1)
    
    masked_pointcloud = []
    masked_color = []

    for j in range(len(confidence_mask)):
        if confidence_mask[j]:
            masked_pointcloud.append(pointcloud[j])
            masked_color.append(color[j])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(masked_pointcloud)
    pcd.colors = o3d.utility.Vector3dVector(masked_color)
    o3d.io.write_point_cloud("{DATA_PATH}/pointclouds/pointcloud{i}.ply".format(DATA_PATH=DATA_PATH, i=i), pcd)

In [14]:
#Create transform file

img = Image.open(images_array[0])
width, height = img.size
RESCALE_FACTOR  = width/512

transforms = {}
transforms["camera_model"] = "OPENCV"
if USE_COMMON_INTRINSICS:
    intrinsic_mean = intrinsics.mean(dim=0)
    transforms["fl_x"] = intrinsic_mean[0,0].item() * RESCALE_FACTOR
    transforms["fl_y"] = intrinsic_mean[1,1].item() * RESCALE_FACTOR
    transforms["w"] = width 
    transforms["h"] = height 
    transforms["cx"] = intrinsic_mean[0,2].item() * RESCALE_FACTOR
    transforms["cy"] = intrinsic_mean[1,2].item() * RESCALE_FACTOR

transforms["frames"] = []

OPENGL = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])

for i in range(len(poses)):
    if not((confidence_masks[i]==0).all()):
        frame = {}
        frame["file_path"] = "/".join(images_array[i].split("/")[-2:])
        #frame["transform_matrix"] = np.linalg.inv(poses[i].detach().cpu().numpy()).tolist()
        frame["transform_matrix"] = np.dot(poses[i].detach().cpu().numpy(),OPENGL).tolist()
        frame["mask_path"] = "/".join(masks_array[i].split("/")[-2:])
        transforms["frames"].append(frame)
        
        if not USE_COMMON_INTRINSICS:
            frame["fl_x"] = intrinsics[i,0,0].item() * RESCALE_FACTOR
            frame["fl_y"] = intrinsics[i,1,1].item() * RESCALE_FACTOR
            frame["cx"] = intrinsics[i,0,2].item() * RESCALE_FACTOR
            frame["cy"] = intrinsics[i,1,2].item() * RESCALE_FACTOR
            img = Image.open(images_array[i])
            width, height = img.size
            transforms["w"] = width 
            transforms["h"] = height 

#Save transform file
with open("{}/transforms.json".format(DATA_PATH), 'w') as f:
    json.dump(transforms, f, indent=4)

In [15]:
import roma

all_poses = torch.stack(list(scene.im_poses))
Q = all_poses[:,:4]
Q = torch.nn.functional.normalize(Q, p=2, dim=1)
T = signed_expm1(all_poses[:,4:7])
tf = roma.RigidUnitQuat(Q, T).normalize().to_homogeneous()

OPENGL = torch.tensor([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]], dtype=torch.float32).to(device)

tf = torch.matmul(tf, OPENGL)

tf = roma.RigidUnitQuat(Q, T).normalize()
print(tf.linear[:,0]/tf.linear[:,3])

tensor([ 0.0698,  0.0579,  0.0453,  0.0311,  0.0065, -0.0121, -0.0283, -0.0453,
        -0.0648, -0.0892, -0.1249, -0.4949,  0.2868,  0.2504,  0.2244,  0.2068,
         0.1933,  0.1815,  0.1708,  0.1618,  0.1518,  0.1398,  0.1449,  0.1325,
         0.1216], device='cuda:0', grad_fn=<DivBackward0>)
